# Prev

## include

In [1]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/run_event.jl")

path_princ = ".results_2"

".results_2"

In [2]:
function create_dir(name)
    path = split(name,"/")
    
    if !(path[size(path,1)] in readdir(join(path[1:size(path,1)-1],"/")))
        mkdir(name)
    end
end

create_dir (generic function with 1 method)

In [3]:
cont_par  = 0.2
force_par = 0.0005

create_dir(path_princ*"/cont_par($(cont_par))")
create_dir(path_princ*"/cont_par($(cont_par))/force_par($(force_par))")

## model

In [4]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "$(cont_par)_$(force_par)",
        path_output = path_princ*"/cont_par($(cont_par))/force_par($(force_par))/",
        d_saved = 0.5
    ) 
)

  0.009273 seconds (16.46 k allocations: 1.150 MiB)


ModelSet(TimeModel(150000.0, 0.5, 100, 50), InputModel(0.8, "../../data/init/Sphere"), OutputModel("0.2_0.0005", ".results_2/cont_par(0.2)/force_par(0.0005)/", 0.5))

# Run

## Some Founded Parameters

### Cubic

In [5]:
# # Complete (Oriola Contractile Model) -> Normal Vectors of distance between cells without area
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.162);
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.166);
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.164);

## Fusion Agg

In [6]:
# Run Model
Par1 ,Par2 = Cubic(force_par,2.0,3.0), ContractilePar(cont_par,pi/4,0.08,1.0)
# Par1 ,Par2 = Cubic(4.2,2.0,3.0), ContractilePar(0.267,pi/4,0.08,1.0)

# ContractilePar(0.02);
# RunFusionAggregates(model::ModelSet, Par1, Par2, 15)
size_agg = 15

15

In [7]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(size_agg).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.0005, 2.0, 3.0), ContractilePar{Float64}(0.2, 0.7853981633974483, 0.08, 1.0)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], (-14.5f0, 14.5f0)), AggSimulation(AggLimit(29.0f0, Bool[0]), AggParameter(Cubic{C

In [8]:
run_test(agg, model,"Run One Aggregate", false, false)

Run One Aggregate...   0%|                               |  ETA: 24.73 days

Run One Aggregate...   3%|█                              |  ETA: 0:07:33

Run One Aggregate...   4%|█▏                             |  ETA: 0:06:39

Run One Aggregate...   4%|█▎                             |  ETA: 0:06:10

Run One Aggregate...   4%|█▍                             |  ETA: 0:05:42

...

Run One Aggregate...  99%|██████████████████████████████▋|  ETA: 0:00:01

Run One Aggregate...  99%|██████████████████████████████▊|  ETA: 0:00:00

Run One Aggregate... 100%|██████████████████████████████▉|  ETA: 0:00:00

Run One Aggregate... 100%|███████████████████████████████|  ETA: 0:00:00

Run One Aggregate... 100%|███████████████████████████████| Time: 0:00:48


In [9]:
agg.Simulation.Output.θr_data

51-element Vector{Any}:
 [3.3423333800276356 15.21151435554223; 2.9593567505369167 15.157040534619608; … ; 6.112307983022465 14.426012221542663; 0.1512627721384232 14.380109280057528]
 [3.0592994531508966 14.31483748247086; 3.0133456103298975 14.38452485077231; … ; 0.09606289014336222 12.486713896904659; 0.11842653306929707 12.516814647604875]
 [3.0410511486248817 15.716486325182146; 3.0257338535454 15.742658198067419; … ; 3.219516827049176 12.558667296972775; 0.09226316586552241 12.57403760504517]
 [6.250589549327307 16.1398235372711; 6.235851752329172 16.149337787819682; … ; 6.246875645463404 12.857494911956072; 6.2780442824474685 12.849190039417556]
 [3.164948324904543 16.260171633774352; 3.1505189230209805 16.25638461226573; … ; 3.2024063573337544 13.4877421545823; 0.07915971934386978 13.50510008880919]
 [3.2151465901253973 15.443020751907808; 3.2151465901253973 15.443020751907808; … ; 3.184944136457847 13.378886974271587; 3.184944136457847 13.378886974271587]
 [3.1475783544825577 

In [10]:
if agg.Simulation.Limit.break_sim == CuArray([false])
    position=agg.Position

    # Run fusion of two aggregates
    agg = nothing
    agg = FusionAggregate(
        [AggType("HEK_1", InteractionPar(Par1, Par2),position)], 
        model
    )
    run_test(agg, model, "Fusion of Two Aggregates", true, true)
elseif agg.Simulation.Limit.break_sim == CuArray([true])
    println("Breaking the Simulation (NaN Value or Bigger Values)")
end

display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

Fusion of Two Aggregates...   0%|                        |  ETA: 16:29:23

Fusion of Two Aggregates...   0%|                        |  ETA: 0:03:26

Fusion of Two Aggregates...   0%|▏                       |  ETA: 0:02:05

Fusion of Two Aggregates...   1%|▏                       |  ETA: 0:01:40

Fusion of Two Aggregates...   1%|▎                       |  ETA: 0:01:24

...

Fusion of Two Aggregates... 100%|████████████████████████| Time: 0:00:45


1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.00254883   -0.0146066    -0.0135494
 -0.0070265    -0.00931344    0.00820228
  0.00217191    0.0149509     0.0103719
 -0.00125626    0.00664918    0.00786128
 -0.0056273     0.0159248    -0.0058067
  0.0065251    -0.00583228   -0.00696156
  0.00226779   -0.00870008    0.00093426
 -0.0301319    -0.00374499   -0.189099
  0.0147203    -0.00323439    0.00190248
 -0.0158737    -0.0112205    -0.0105774
  ⋮                          
  0.106198     -0.0100938     0.172502
  0.0131085     0.0063232     0.00459105
 -0.000108202  -0.00698228    0.00102839
  0.10381       0.166492      0.0343733
 -0.00447482   -0.00518232    0.00671073
  0.00574705    0.00305259    0.00396808
 -0.00797212    0.000275267   0.00761536
  0.00721324   -0.00359964    0.0154562
  0.0133165     0.00383086   -0.00959731

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -18.0423   -3.97123  -4.62422
 -15.9302   -1.62856  -7.06378
 -17.165    -9.08389  -6.71942
 -25.2763    2.92131  -4.74798
 -26.8496   -9.27464  -6.95494
 -23.6767   -1.6246   -7.7922
 -17.9625   -3.91859  -4.4337
 -16.5493   -9.33343  -6.67691
 -41.2241   -6.73557  -0.865803
 -41.6138   -6.71811  -1.45883
   ⋮                  
  32.6613   -2.227    18.2525
  15.7139    6.01108  11.6007
  32.3969   -2.43071  18.1038
  21.1657    3.52473   1.85897
  13.5108   15.8414   15.9669
  32.2128   -1.71739  17.9747
  28.2704   -3.68351   6.8554
   1.66865   2.38663   6.00465
  14.0569   15.9903   15.2019

# Visualization

## θr data

In [11]:
# using Plots
# if agg.Simulation.Limit.break_sim == CuArray([false])
#     p = Progress(convert(Int,model.Time.nₛₐᵥₑ+1),"Animating ...")
#     anim = @animate for t = 1:model.Time.nₛₐᵥₑ+1
#         plot(
#             agg.Simulation.Output.θr_data[t][:,1], 
#             agg.Simulation.Output.θr_data[t][:,2], 
#             title = "t=$((t-1)*model.Time.tₛᵢₘ/model.Time.nₛₐᵥₑ)",
#             seriestype=:scatter,
#             legend = false
#         )
#     end

#     gif(anim, fps=2)
# elseif agg.Simulation.Limit.break_sim == CuArray([true])
#     println("Breaking the Simulation (NaN Value or Bigger Values)")
# end

## xy data

In [12]:
# using Plots
# if agg.Simulation.Limit.break_sim == CuArray([false])
#     p = Progress(convert(Int,model.Time.nₛₐᵥₑ+1),"Animating ...")
#     anim = @animate for t = 1:model.Time.nₛₐᵥₑ+1
#         plot(
#             agg.Simulation.Output.xy_data[t][:,1], 
#             agg.Simulation.Output.xy_data[t][:,2], 
#             title = "t=$((t-1)*model.Time.tₛᵢₘ/model.Time.nₛₐᵥₑ)",
#             seriestype=:scatter,
#             legend = false
#         )
#     end

#     gif(anim, fps=2)
# elseif agg.Simulation.Limit.break_sim == CuArray([true])
#     println("Breaking the Simulation (NaN Value or Bigger Values)")
# end

# Neck - Width

In [13]:
# if agg.Simulation.Limit.break_sim == CuArray([false])
#     display(
#         plot(
#             agg.Simulation.Output.time, 
#             agg.Simulation.Output.width_data', 
#             title = "Width",
#             seriestype=:scatter,
#             legend = false
#         )
#     )
#     display(
#         plot(
#             agg.Simulation.Output.time, 
#             agg.Simulation.Output.neck_data', 
#             title = "Neck",
#             seriestype=:scatter,
#             legend = false
#         )
#     )
# elseif agg.Simulation.Limit.break_sim == CuArray([true])
#     println("Breaking the Simulation (NaN Value or Bigger Values)")
# end